In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db' # путь к файлу
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
WITH

u AS

(SELECT user_id,
          lt_day,
          (CASE
               WHEN lt_day <= 365 THEN 'new'
               WHEN lt_day > 365 THEN 'not_new'
           END) AS is_new,
          age,
          (CASE
               WHEN gender_segment=1 THEN 'female'
               WHEN gender_segment=0 THEN 'male'
           END) AS gender_segment,
          os_name,
          cpe_type_name,
          nps_score,
          (CASE
               WHEN nps_score>=9 THEN 'promoters'
               WHEN nps_score>=7
                    AND nps_score<9 THEN 'passives'
               ELSE 'detractors'
           END) AS nps_group,
          location_id,
          age_gr_id,
          tr_gr_id,
          lt_gr_id
   FROM USER),

l AS

(SELECT country, city, location_id
 FROM location 
),

a_s AS 

(SELECT age_gr_id, title
 FROM age_segment
),

t_s AS

(SELECT tr_gr_id, title
 FROM traffic_segment
),

l_s AS

(SELECT lt_gr_id, title
 FROM lifetime_segment
)

SELECT u.user_id AS user_id,
       u.lt_day AS lt_day,
       u.is_new AS is_new,
       u.age AS age,
       u.gender_segment AS gender_segment,
       u.os_name AS os_name,
       u.cpe_type_name AS cpe_type_name,
       l.country AS country,
       l.city AS city,
       SUBSTR(a_s.title,3) AS age_segment,
       SUBSTR(t_s.title,3) AS traffic_segment,
       SUBSTR(l_s.title,3) AS lifetime_segment,
       u.nps_score AS nps_score,
       u.nps_group AS nps_group
FROM u JOIN l ON u.location_id=l.location_id
JOIN a_s ON u.age_gr_id=a_s.age_gr_id
JOIN t_s ON u.tr_gr_id=t_s.tr_gr_id
JOIN l_s ON u.lt_gr_id=l_s.lt_gr_id

"""

In [4]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,not_new,45.0,female,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,not_new,53.0,male,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,not_new,57.0,male,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,not_new,44.0,female,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,not_new,24.0,male,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters
5,A005O0,5501,not_new,42.0,female,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,detractors
6,A0061R,1236,not_new,45.0,male,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,promoters
7,A009KS,313,new,35.0,male,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,promoters
8,A00AES,3238,not_new,36.0,female,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,promoters
9,A00F70,4479,not_new,54.0,female,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,promoters


In [5]:
df.to_csv('telecomm_csi_tableau.csv')

### Ссылка на дашборд на сайте Tableau Public:

Ссылка на дашборд: <https://public.tableau.com/app/profile/zhanna6775/viz/prefab_project_2/Dashboard1?publish=yes>

### Ссылка на pdf-файл с презентацией:

Ссылка на презентацию: <https://drive.google.com/file/d/1-Kdb39IfGpjcrEiC3SLObi8I-eMItxVt/view?usp=sharing>